In [1]:
%%capture
!pip install kornia kornia_moons==0.2.0 --no-deps
!pip install opencv-python --upgrade

# A function to convert pairs of LAFs to affine correspondences (ACs)
def get_affine_correspondences(lafs1, lafs2, tentatives):
    # Initialize the ACs
    ACs = np.zeros((len(tentatives), 8))
    row = 0
    # Iterate through all tentative correspondences and them to the matrix
    for m in tentatives:
        # The LAF in the source image
        LAF1 = lafs1[m.queryIdx]
        # The LAF in the destination image
        LAF2 = lafs2[m.trainIdx]
        # Calculating the local affine transformation as A = LAF2 * inv(LAF1)
        A = np.matmul(LAF2[:, :2], np.linalg.inv(LAF1[:, :2]))

        # Saving the coordinates in the ACs matrix
        ACs[row, 0] = LAF1[0, 2]
        ACs[row, 1] = LAF1[1, 2]
        ACs[row, 2] = LAF2[0, 2]
        ACs[row, 3] = LAF2[1, 2]
        ACs[row, 4] = A[0, 0]
        ACs[row, 5] = A[0, 1]
        ACs[row, 6] = A[1, 0]
        ACs[row, 7] = A[1, 1]
        row += 1

    return np.float32(ACs)

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pygcransac
import os
import h5py
from copy import deepcopy
import kornia as K
import kornia.feature as KF
import torch
from kornia_moons.feature import *
from time import time
from gcransac_parameter_types import *

# Function to load the image into a pytorch tensor
def load_torch_image(fname):
    img = K.image_to_tensor(cv2.imread(fname), False).float() / 255.
    img = K.color.bgr_to_rgb(img)
    return img
    
# Deciding about the device used. Prefer CUDA if available.
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

print(device)

# The number of keypoints to be detected
desired_kpts = 8000
        
# Detecting DoG + HardNet + AffNet features
kornia_cv2dogaffnet = OpenCVDetectorWithAffNetKornia(cv2.SIFT_create(int(desired_kpts * 1.3)), make_upright=True, max_kpts=desired_kpts)
dogaffnethardnet = KF.LocalFeature(kornia_cv2dogaffnet, KF.LAFDescriptor(KF.HardNet(True))).eval()

scenes = [ "OldHospital", "KingsCollege"] # "ShopFacade", "StMarysChurch",
path_dataset = "/media/hdd3tb/datasets/absolute_pose/"

for scene in scenes:
    path_pairs = path_dataset + scene + "/" + scene + "_top20.txt"
    path_scene = path_dataset + scene + "/" + scene

    database = h5py.File(os.path.join(path_dataset, scene + '_ACs' + str(desired_kpts) + '_upright.h5'), 'a')

    with open(path_pairs, 'r') as f:
        while True:
            line = f.readline()
            if not line: 
                break
            
            source_image_name = line.split(' ')[0]
            destination_image_name = line.split(' ')[1][:-1]

            # Detect features in the source image
            if source_image_name + "-descs" not in database.keys():
                img1 = load_torch_image(os.path.join(path_scene, source_image_name)) 
                img1.to(device)
                lafs1, r1, descs1 = dogaffnethardnet(img1)

                descs1np = np.squeeze(descs1.cpu().detach().numpy())
                lafs1np = np.squeeze(lafs1.cpu().detach().numpy())
                r1np = np.squeeze(r1.cpu().detach().numpy())

                database.create_dataset(source_image_name + "-descs", data=descs1np)
                database.create_dataset(source_image_name + "-lafs", data=lafs1np)
            else:
                descs1np = np.array(database[source_image_name + "-descs"])
                lafs1np = np.array(database[source_image_name + "-lafs"])

            # Detect features in the source image
            if destination_image_name + "-descs" not in database.keys():
                img2 = load_torch_image(os.path.join(path_scene, destination_image_name)) 
                img2.to(device)
                lafs2, r2, descs2 = dogaffnethardnet(img2)

                descs2np = np.squeeze(descs2.cpu().detach().numpy())
                lafs2np = np.squeeze(lafs2.cpu().detach().numpy())
                r2np = np.squeeze(r2.cpu().detach().numpy())
                
                database.create_dataset(destination_image_name + "-descs", data=descs2np)
                database.create_dataset(destination_image_name + "-lafs", data=lafs2np)
            else:
                descs2np = np.array(database[destination_image_name + "-descs"])
                lafs2np = np.array(database[destination_image_name + "-lafs"])

            if source_image_name + "-" + destination_image_name not in database.keys():
                # Initialize a brute-force matcher to match the descriptors
                bf = cv2.BFMatcher()

                # Applying brute-force matcher
                matches = bf.knnMatch(descs1np, descs2np, k = 2)
                
                # Apply the SNN ratio test
                snn_ratios = []
                tentatives = []
                for m, n in matches:
                    if n.distance == 0:
                        continue
                    tentatives.append(m)
                    snn_ratios.append(m.distance / n.distance)

                # Sort the keypoints based on the SNN ratio.
                # This is used in many samplers, e.g., PROSAC, NG-RANSAC's, AR-Sampler
                sorted_indices = np.argsort(snn_ratios)
                tentatives = list(np.array(tentatives)[sorted_indices])
                snn_ratios = list(np.array(snn_ratios)[sorted_indices])

                correspondences = get_affine_correspondences(lafs1np, lafs2np, tentatives)
                final_correspondences = np.zeros((len(tentatives), 9))
                final_correspondences[:,0:8] = correspondences
                final_correspondences[:, 8] = np.array(snn_ratios)
                
                database.create_dataset(source_image_name + "-" + destination_image_name, data=final_correspondences)

                print(f"{source_image_name} {destination_image_name} - {len(tentatives)} tentative correspondences are found.")
                database.flush()
    database.close()

cuda
seq3/frame00008.png seq2/frame00007.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00008.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00005.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00001.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00002.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00003.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00004.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00009.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00010.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00231.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00228.png - 8000 tentative correspondences are found.
seq3/frame00008.png seq2/frame00011.png - 8000 tentative correspondence

: 

: 